In [1]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import Dataset
import torch
from torch.nn import CrossEntropyLoss

c:\Users\zlata\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def balance_dataset(df, target_col):
    a, b = sorted(dict(df[target_col].value_counts()).items())
    if (a[1] > b[1]):
        tmp = df.loc[df[target_col] == 1].sample(n = abs(a[1] - b[1]))
    else:
        tmp = df.loc[df[target_col] == 0].sample(n = abs(a[1] - b[1]))
    df = pd.concat([df, tmp], ignore_index=True)
    return df

def remove_some(s, lst = ['игра', 'это', 'всё', 'который', 'весь']):
    for i in lst:
        s = s.replace(i, '')
    return s

def drop_trash(df):
    df = df.dropna()
    empty = df[(df.text == '') | (df.text == ' ')]
    df = df.drop(empty.index)
    df = df.drop(df[df.text.str.split().apply(len) == 1].text.index)
    df.text = df.text.apply(remove_some)
    df = df.drop_duplicates(subset='text', keep="last")
    return df

In [3]:
file_path = "cleaned_bin_df_politics.csv"
df = pd.read_csv(file_path, index_col=0)
df = balance_dataset(drop_trash(df), 'label')
df.head()

,text,label,assessment
0,многий прохождение прочитать титр узнать имя...,1.0,1.0
1,узнать лента новость человек следить отечеств...,1.0,0.0
2,пройти российский студия разработчик описывать...,1.0,0.0
3,купить подборка порыв поддержать отечественны...,1.0,0.0
4,совет проверить товарищ третий день купить под...,1.0,1.0


In [4]:
df.label.value_counts()

label
1.0    7081
0.0    7081
Name: count, dtype: int64

In [5]:
X = df['text']
y = df['label'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

0        многий  прохождение  прочитать титр узнать имя...
1        узнать  лента новость человек следить отечеств...
2        пройти российский студия разработчик описывать...
3        купить  подборка порыв поддержать отечественны...
4        совет проверить товарищ третий день купить под...
                               ...                        
14157    ваш флип керкоп похожий чорт сатанист просто п...
14158    история нс год назад выбрать ноутбук город ост...
14159    россия рулить флаг азербайджан алый сердце фла...
14160    проходить  наверное переставать надоедать  пог...
14161    слушать неплохой идея обзор посветить обзор се...
Name: text, Length: 14162, dtype: object

In [6]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from safetensors import safe_open
tensors = {}
with safe_open(r"C:\Users\zlata\Documents\parser_ign\models\my_model_v2\model.safetensors", framework="pt", device='cpu') as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

In [10]:
model.load_state_dict(tensors)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [11]:
train_data = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train}))
test_data = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test}))

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|██████████| 2833/2833 [00:06<00:00, 469.61 examples/s]


In [19]:
training_args = TrainingArguments(
    output_dir='./results_new_v2',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    logging_steps=10,
    report_to="none"  
)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    f1 = f1_score(labels, pred)
    return {'f1': f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

  1%|          | 10/1417 [01:36<3:44:47,  9.59s/it]

{'loss': 0.0004, 'grad_norm': 0.183873251080513, 'learning_rate': 4.964714184897671e-05, 'epoch': 0.01}


  1%|▏         | 20/1417 [03:12<3:45:07,  9.67s/it]

{'loss': 0.0025, 'grad_norm': 0.003003583522513509, 'learning_rate': 4.929428369795343e-05, 'epoch': 0.01}


  2%|▏         | 30/1417 [04:50<3:47:37,  9.85s/it]

...

100%|██████████| 1417/1417 [3:26:46<00:00,  4.82s/it]

{'eval_loss': 0.8440388441085815, 'eval_f1': 0.8728813559322034, 'eval_runtime': 1303.3926, 'eval_samples_per_second': 2.174, 'eval_steps_per_second': 0.272, 'epoch': 1.0}


100%|██████████| 1417/1417 [3:26:48<00:00,  8.76s/it]

{'train_runtime': 12408.1306, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.114, 'train_loss': 0.042522864496508245, 'epoch': 1.0}


TrainOutput(global_step=1417, training_loss=0.042522864496508245, metrics={'train_runtime': 12408.1306, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.114, 'total_flos': 83542390831104.0, 'train_loss': 0.042522864496508245, 'epoch': 1.0})

In [21]:
eval_result = trainer.evaluate()
f1_rubert_tiny2 = eval_result['eval_f1']
print(f'F1-score для rubert-tiny2: {f1_rubert_tiny2:.4f}')


100%|██████████| 355/355 [21:44<00:00,  3.68s/it]

F1-score для rubert-tiny2: 0.8729


In [22]:
model.save_pretrained('./models/my_model_v3')